In [ ]:
!pip install -q opendatasets
import opendatasets as od
od.download(
    "https://www.kaggle.com/competitions/wind-turbine-damage-challenges")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: visvaldas
Your Kaggle Key: ··········
Extracting archive ./wind-turbine-damage-challenges/wind-turbine-damage-challenges.zip to ./wind-turbine-damage-challenges


In [ ]:
!pip install -U openmim
!mim install mmengine
!pip install mmcv==2.2.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.4/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
  Using cached mmengine-0.10.7-py3-none-any.whl.metadata (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.5 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.4/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


1. Создать датасет в mmdet/datasets, поменять classes и palette, задать импорт в init.
2. mmdetection/datasets/coco_detection - поменять пути, задать DefectDataset
3. mmdetection/config/... - поменять число классов с 80 на 5
4. mmdet - поменять макс. версию на 2.2.1
5. /usr/local/lib/...mmengine - 347 - загрузить веса

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp /content/drive/MyDrive/mmdetection.zip /content/mmdetection.zip
!unzip mmdetection.zip
%cd mmdetection
!pip install -v -e .

Archive:  mmdetection.zip
   creating: mmdetection/
   creating: mmdetection/.circleci/
  inflating: mmdetection/.circleci/config.yml  
   creating: mmdetection/.circleci/docker/
  inflating: mmdetection/.circleci/docker/Dockerfile  
  inflating: mmdetection/.circleci/test.yml  
   creating: mmdetection/.dev_scripts/
  inflating: mmdetection/.dev_scripts/batch_test_list.py  
  inflating: mmdetection/.dev_scripts/batch_train_list.txt  
  inflating: mmdetection/.dev_scripts/benchmark_filter.py  
  inflating: mmdetection/.dev_scripts/benchmark_full_models.txt  
  inflating: mmdetection/.dev_scripts/benchmark_inference_fps.py  
  inflating: mmdetection/.dev_scripts/benchmark_options.py  
  inflating: mmdetection/.dev_scripts/benchmark_test.py  
  inflating: mmdetection/.dev_scripts/benchmark_test_image.py  
  inflating: mmdetection/.dev_scripts/benchmark_train.py  
  inflating: mmdetection/.dev_scripts/benchmark_train_models.txt  
  inflating: mmdetection/.dev_scripts/benchmark_valid_flops

In [ ]:
!python mmdetection/tools/train.py /content/mmdetection/configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py

In [ ]:
# !cp '/content/drive/MyDrive/epoch_12 (1).pth' /content/epoch_12.pth

In [ ]:
!python tools/test.py configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py /content/epoch_12.pth --out /content/result.pkl --show --show-dir /content/show

In [ ]:
import glob
dataset = glob.glob("/content/wind-turbine-damage-challenges/Wind Turbine damage/test/*.jpg")

In [ ]:
from mmdet.apis import DetInferencer

config_file = '/content/mmdetection/configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py'
checkpoint_file = '/content/epoch_12.pth'

inferencer = DetInferencer(model=config_file, weights=checkpoint_file, device='cuda:0')
results = inferencer('/content/wind-turbine-damage-challenges/Wind Turbine damage/test/')

Loads checkpoint by local backend from path: /content/epoch_12.pth


Output()

In [ ]:
results['predictions']

In [ ]:
import pickle


with open('result.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

In [ ]:
!pip install ultralytics

In [ ]:
import os
import json
from tqdm import tqdm

def coco_to_yolo(coco_path, output_dir, images_dir):
    with open(coco_path) as f:
        coco = json.load(f)

    os.makedirs(output_dir, exist_ok=True)

    # Создаём словарь id → category_id
    category_map = {cat['id']: idx for idx, cat in enumerate(coco['categories'])}

    # Создаём словарь image_id → file_name, size
    image_info = {img['id']: img for img in coco['images']}

    for ann in tqdm(coco['annotations']):
        img = image_info[ann['image_id']]
        width, height = img['width'], img['height']
        x, y, w, h = ann['bbox']
        xc = (x + w / 2) / width
        yc = (y + h / 2) / height
        w /= width
        h /= height
        class_id = category_map[ann['category_id']]

        label_path = os.path.join(output_dir, os.path.splitext(img['file_name'])[0] + ".txt")
        with open(label_path, 'a') as f:
            f.write(f"{class_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
coco_to_yolo('/content/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json', '/content/datasets/wind-turbine-damage-challenges/labels/train', '/content/wind-turbine-damage-challenges/Wind Turbine damage/train/')

100%|██████████| 9150/9150 [00:00<00:00, 40925.71it/s]


In [ ]:
coco_to_yolo('/content/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json', '/content/datasets/wind-turbine-damage-challenges/labels/valid', '/content/wind-turbine-damage-challenges/Wind Turbine damage/valid/')

100%|██████████| 2619/2619 [00:00<00:00, 28029.17it/s]


In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/train' '/content/datasets/wind-turbine-damage-challenges/images/train'

In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/valid' '/content/datasets/wind-turbine-damage-challenges/images/val'

In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/test' '/content/datasets/wind-turbine-damage-challenges/images/test'

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # или yolov8s.pt
model.train(data='dataset.yaml', epochs=50, imgsz=640)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 363MB/s]

Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=True, save

100%|██████████| 755k/755k [00:00<00:00, 103MB/s]

...

                   all        255       2447      0.216      0.116      0.113      0.039



50 epochs completed in 0.958 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:12<00:00,  1.61s/it]


                   all        255       2447      0.215      0.116      0.113      0.039
             lightning        122       2064      0.393      0.131       0.13     0.0537
    lightning_receptor          2          2          0          0          0          0
         missing_teeth          6          6          0          0          0          0
                 patch         79        375      0.468      0.333      0.323      0.102
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c6b0c7e6090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
import pandas as pd

results = model.predict(source='datasets/wind-turbine-damage-challenges/images/test', save=False)

rows = []
for r in results:
    image_id = os.path.basename(r.path)
    for box in r.boxes:
        class_id = int(box.cls)
        conf = float(box.conf)
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        rows.append([image_id, x1, y1, x2, y2, class_id, conf])

df = pd.DataFrame(rows, columns=['image_id', 'x1', 'y1', 'x2', 'y2', 'class_id', 'confidence'])
df.to_csv("submission.csv", index=False)


In [ ]:
!zip -r * content.zip

In [ ]:
!cp datasets.zip drive/MyDrive

In [ ]:
import json
import pandas as pd

# Путь к файлу с результатами инференса
results_path = "farazlar.txt"  # или .json, если вы сохраняете в этом формате
output_csv = "submission.csv"

# Загрузка результатов
with open(results_path, "r", encoding="utf-8") as f:
    results = json.load(f)

# Составляем submission
submission_data = []

i = 0

for image_id, result in enumerate(results):
    labels = result.get("labels", [])
    scores = result.get("scores", [])
    bboxes = result.get("bboxes", [])

    for label, score, bbox in zip(labels, scores, bboxes):
        xmin, ymin, xmax, ymax = bbox
        submission_data.append({
            "image_id": image_id,
            "xmin": round(xmin, 2),
            "ymin": round(ymin, 2),
            "xmax": round(xmax, 2),
            "ymax": round(ymax, 2),
            "label": label,
            "score": round(score, 4)
        })

# Преобразуем в DataFrame и сохраняем
df = pd.DataFrame(submission_data)
df.to_csv(output_csv, index=False)

print(f"Сабмит сохранён в {output_csv}")

Сабмит сохранён в submission.csv


In [ ]:
len(results)

151